# Natural Language Preprocessing

In this notebook, we clean up a dataset of natural language data (books from Project Gutenberg) and use word2vec to embed the language in word vectors.

**N.B.:** Some, all or none of these preprocessing steps may be helpful to a given downstream application. 

### Load Dependencies

In [2]:
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import *
nltk.download('gutenberg')
nltk.download('punkt')
nltk.download('stopwords')

import string

import gensim
from gensim.models.phrases import Phraser, Phrases
from gensim.models.word2vec import Word2Vec

import spacy # for a lemmatization example

from sklearn.manifold import TSNE

import pandas as pd
from bokeh.io import output_notebook, output_file
from bokeh.plotting import show, figure

[nltk_data] Downloading package gutenberg to C:\Users\ANILESH
[nltk_data]     PRAJAPATI\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\ANILESH
[nltk_data]     PRAJAPATI\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\ANILESH
[nltk_data]     PRAJAPATI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Load Data

In [5]:
from nltk.corpus import gutenberg

In [6]:
# a convenient method that handles newlines , as well as tokenizing sentences and words in one shot
gberg_sents = gutenberg.sents()

In [7]:
gberg_sents[0:6]

[['[', 'Emma', 'by', 'Jane', 'Austen', '1816', ']'],
 ['VOLUME', 'I'],
 ['CHAPTER', 'I'],
 ['Emma',
  'Woodhouse',
  ',',
  'handsome',
  ',',
  'clever',
  ',',
  'and',
  'rich',
  ',',
  'with',
  'a',
  'comfortable',
  'home',
  'and',
  'happy',
  'disposition',
  ',',
  'seemed',
  'to',
  'unite',
  'some',
  'of',
  'the',
  'best',
  'blessings',
  'of',
  'existence',
  ';',
  'and',
  'had',
  'lived',
  'nearly',
  'twenty',
  '-',
  'one',
  'years',
  'in',
  'the',
  'world',
  'with',
  'very',
  'little',
  'to',
  'distress',
  'or',
  'vex',
  'her',
  '.'],
 ['She',
  'was',
  'the',
  'youngest',
  'of',
  'the',
  'two',
  'daughters',
  'of',
  'a',
  'most',
  'affectionate',
  ',',
  'indulgent',
  'father',
  ';',
  'and',
  'had',
  ',',
  'in',
  'consequence',
  'of',
  'her',
  'sister',
  "'",
  's',
  'marriage',
  ',',
  'been',
  'mistress',
  'of',
  'his',
  'house',
  'from',
  'a',
  'very',
  'early',
  'period',
  '.'],
 ['Her',
  'mother',
  'h

In [8]:
gberg_sents[4][14]

'father'

## Iteratively Preprocess a Sentence

### A Tokenized Sentence:


In [9]:
gberg_sents[4]

['She',
 'was',
 'the',
 'youngest',
 'of',
 'the',
 'two',
 'daughters',
 'of',
 'a',
 'most',
 'affectionate',
 ',',
 'indulgent',
 'father',
 ';',
 'and',
 'had',
 ',',
 'in',
 'consequence',
 'of',
 'her',
 'sister',
 "'",
 's',
 'marriage',
 ',',
 'been',
 'mistress',
 'of',
 'his',
 'house',
 'from',
 'a',
 'very',
 'early',
 'period',
 '.']

### To Lowercase:

In [10]:
[w.lower() for w in gberg_sents[4]]

['she',
 'was',
 'the',
 'youngest',
 'of',
 'the',
 'two',
 'daughters',
 'of',
 'a',
 'most',
 'affectionate',
 ',',
 'indulgent',
 'father',
 ';',
 'and',
 'had',
 ',',
 'in',
 'consequence',
 'of',
 'her',
 'sister',
 "'",
 's',
 'marriage',
 ',',
 'been',
 'mistress',
 'of',
 'his',
 'house',
 'from',
 'a',
 'very',
 'early',
 'period',
 '.']

### Remove Stopwords & Punctuation:

In [11]:
stpwrds = stopwords.words('english') + list(string.punctuation)

In [12]:
stpwrds

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [13]:
[w.lower() for w in gberg_sents[4] if w.lower() not in stpwrds]

['youngest',
 'two',
 'daughters',
 'affectionate',
 'indulgent',
 'father',
 'consequence',
 'sister',
 'marriage',
 'mistress',
 'house',
 'early',
 'period']

### Stem Words:

In [14]:
stemmer = PorterStemmer()

In [15]:
[stemmer.stem(w.lower()) for w in gberg_sents[4] if w.lower() not in stpwrds]

['youngest',
 'two',
 'daughter',
 'affection',
 'indulg',
 'father',
 'consequ',
 'sister',
 'marriag',
 'mistress',
 'hous',
 'earli',
 'period']

### A Lemmatization Example:

In [19]:
nlp = spacy.load("en_core_web_sm")

In [20]:
gutenberg.raw()[291:477]

"She was the youngest of the two daughters of a most affectionate,\nindulgent father; and had, in consequence of her sister's marriage,\nbeen mistress of his house from a very early period."

In [21]:
spacy_doc = nlp(gutenberg.raw()[291:477])

In [22]:
[w.lemma_ for w in spacy_doc]

['she',
 'be',
 'the',
 'young',
 'of',
 'the',
 'two',
 'daughter',
 'of',
 'a',
 'most',
 'affectionate',
 ',',
 '\n',
 'indulgent',
 'father',
 ';',
 'and',
 'have',
 ',',
 'in',
 'consequence',
 'of',
 'her',
 'sister',
 "'s",
 'marriage',
 ',',
 '\n',
 'be',
 'mistress',
 'of',
 'his',
 'house',
 'from',
 'a',
 'very',
 'early',
 'period',
 '.']

## Handle Bigram Collecations:

In [23]:
phrases = Phrases(gberg_sents) # train detector

In [24]:
bigram = Phraser(phrases) # create a more efficient Phraser object for transform sentences

In [25]:
bigram.phrasegrams # output count and  score of each bigram.

{'two_daughters': 11.966813731181547,
 'her_sister': 17.7960829227865,
 "'_s": 31.066242737744524,
 'very_early': 11.01214147275924,
 'Her_mother': 13.529425062715127,
 'long_ago': 63.22343628984789,
 'more_than': 29.023584433996874,
 'had_been': 22.306024648925288,
 'an_excellent': 39.063874851750626,
 'Miss_Taylor': 453.75918026073305,
 'very_fond': 24.134280468850747,
 'passed_away': 12.35053642325912,
 'too_much': 31.376002029426687,
 'did_not': 11.72841621714281,
 'any_means': 14.096964108090186,
 'wedding_-': 17.4695197740113,
 'Her_father': 13.129571562488772,
 'after_dinner': 21.528548116881705,
 'self_-': 47.79018053120332,
 'sixteen_years': 107.0461671612265,
 'five_years': 40.128755673408115,
 'years_old': 54.735425236061104,
 'seven_years': 52.59411150244507,
 'each_other': 79.4168405322873,
 'a_mile': 12.783091600264584,
 'must_be': 10.229989650632808,
 'difference_between': 220.52537305244678,
 'could_not': 10.870983286982371,
 'having_been': 11.53801833156938,
 'miles_of

In [26]:
tokenized_sentence = "Jon lives in New York City".split()

In [27]:
tokenized_sentence

['Jon', 'lives', 'in', 'New', 'York', 'City']

In [28]:
bigram[tokenized_sentence]

['Jon', 'lives', 'in', 'New_York', 'City']

## Preprocess The Corpus

In [29]:
# as in Maas et al. (2001):
# - Leave in stop word ("indicative of sentiment")
# - no stemming ("model Learns similar representations of word of the same stem when data suggest it")

lower_sents = []
for s in gberg_sents:
    lower_sents.append([w.lower() for w in s if w.lower() not in list (string.punctuation)])

In [30]:
lower_sents[0:5]

[['emma', 'by', 'jane', 'austen', '1816'],
 ['volume', 'i'],
 ['chapter', 'i'],
 ['emma',
  'woodhouse',
  'handsome',
  'clever',
  'and',
  'rich',
  'with',
  'a',
  'comfortable',
  'home',
  'and',
  'happy',
  'disposition',
  'seemed',
  'to',
  'unite',
  'some',
  'of',
  'the',
  'best',
  'blessings',
  'of',
  'existence',
  'and',
  'had',
  'lived',
  'nearly',
  'twenty',
  'one',
  'years',
  'in',
  'the',
  'world',
  'with',
  'very',
  'little',
  'to',
  'distress',
  'or',
  'vex',
  'her'],
 ['she',
  'was',
  'the',
  'youngest',
  'of',
  'the',
  'two',
  'daughters',
  'of',
  'a',
  'most',
  'affectionate',
  'indulgent',
  'father',
  'and',
  'had',
  'in',
  'consequence',
  'of',
  'her',
  'sister',
  's',
  'marriage',
  'been',
  'mistress',
  'of',
  'his',
  'house',
  'from',
  'a',
  'very',
  'early',
  'period']]

In [32]:
lower_bigram = Phraser(Phrases(lower_sents))

In [33]:
lower_bigram.phrasegrams # miss taylor , mr woodhouse , mr weston

{'two_daughters': 11.080802900992637,
 'her_sister': 16.93971298099339,
 'very_early': 10.516998773665177,
 'her_mother': 10.70812618607742,
 'long_ago': 59.22644201533601,
 'more_than': 28.529926612065935,
 'had_been': 21.583193129694834,
 'an_excellent': 37.41859680854167,
 'sixteen_years': 131.42913000977518,
 'miss_taylor': 420.4340982546865,
 'mr_woodhouse': 104.19907841850323,
 'very_fond': 24.185726346489627,
 'passed_away': 11.751473221742694,
 'too_much': 30.363090173835406,
 'did_not': 10.846196223896685,
 'any_means': 14.294148100212627,
 'after_dinner': 18.60737125272944,
 'mr_weston': 91.63290824201266,
 'five_years': 37.66428596665674,
 'years_old': 48.59909444619029,
 'seven_years': 50.3345604292756,
 'each_other': 71.31277029783762,
 'well_informed': 14.185028016786626,
 'a_mile': 11.700110753652233,
 'difference_between': 207.86784241868986,
 'mrs_weston': 180.6778969011602,
 'could_not': 10.213333164207082,
 'having_been': 10.723750443105281,
 'sixteen_miles': 105.040

In [35]:
lower_bigram["jon lives in new york city".split()]

['jon', 'lives', 'in', 'new_york', 'city']

In [36]:
lower_bigram = Phraser(Phrases(lower_sents, min_count=32, threshold=64))
lower_bigram.phrasegrams

{'miss_taylor': 156.44059469941823,
 'mr_woodhouse': 82.04651843976633,
 'mr_weston': 75.87438262077481,
 'mrs_weston': 160.6848509325892,
 'great_deal': 93.36368125424357,
 'mr_knightley': 161.74131790625913,
 'miss_woodhouse': 229.03802722366902,
 'years_ago': 74.31594785893046,
 'mr_elton': 121.3990121932397,
 'dare_say': 89.94000515807346,
 'frank_churchill': 1316.4456593286038,
 'miss_bates': 276.3958829169252,
 'drawing_room': 84.91494947493561,
 'mrs_goddard': 143.57843432545658,
 'miss_smith': 73.03442128232508,
 'few_minutes': 204.16834974753786,
 'john_knightley': 83.03755747111268,
 'don_t': 250.30957446808512,
 'good_natured': 88.69936184891343,
 'few_moments': 107.77584531675087,
 'thousand_pounds': 166.51834523092802,
 'o_clock': 89.14789088153574,
 'jane_fairfax': 654.556591758761,
 'miss_fairfax': 196.19987447261062,
 'ma_am': 157.25846601094193,
 'mrs_elton': 93.08931456265867,
 'forty_years': 90.60220877269607,
 'cut_off': 129.60290535032792,
 'ten_thousand': 84.00099

In [37]:
clean_sents = []
for s in lower_sents:
    clean_sents.append(lower_bigram[s])

In [38]:
clean_sents[0:9]

[['emma', 'by', 'jane', 'austen', '1816'],
 ['volume', 'i'],
 ['chapter', 'i'],
 ['emma',
  'woodhouse',
  'handsome',
  'clever',
  'and',
  'rich',
  'with',
  'a',
  'comfortable',
  'home',
  'and',
  'happy',
  'disposition',
  'seemed',
  'to',
  'unite',
  'some',
  'of',
  'the',
  'best',
  'blessings',
  'of',
  'existence',
  'and',
  'had',
  'lived',
  'nearly',
  'twenty',
  'one',
  'years',
  'in',
  'the',
  'world',
  'with',
  'very',
  'little',
  'to',
  'distress',
  'or',
  'vex',
  'her'],
 ['she',
  'was',
  'the',
  'youngest',
  'of',
  'the',
  'two',
  'daughters',
  'of',
  'a',
  'most',
  'affectionate',
  'indulgent',
  'father',
  'and',
  'had',
  'in',
  'consequence',
  'of',
  'her',
  'sister',
  's',
  'marriage',
  'been',
  'mistress',
  'of',
  'his',
  'house',
  'from',
  'a',
  'very',
  'early',
  'period'],
 ['her',
  'mother',
  'had',
  'died',
  'too',
  'long',
  'ago',
  'for',
  'her',
  'to',
  'have',
  'more',
  'than',
  'an',
 

In [39]:
clean_sents[6]

['sixteen',
 'years',
 'had',
 'miss_taylor',
 'been',
 'in',
 'mr_woodhouse',
 's',
 'family',
 'less',
 'as',
 'a',
 'governess',
 'than',
 'a',
 'friend',
 'very',
 'fond',
 'of',
 'both',
 'daughters',
 'but',
 'particularly',
 'of',
 'emma']

## Run word2vec

In [42]:
# max_vocab_size can be used insteas of min_count (which has increased here)
model = Word2Vec(sentences=clean_sents, vector_size=64, 
                sg=1, window=10, epochs=5,
                min_count=10, workers=4)
model.save('clean_gutenberg_model.w2v')

## Explore Model

In [44]:
# skip re_training the model with the next line:
model = gensim.models.Word2Vec.load('clean_gutenberg_model.w2v')

In [45]:
len(model.wv) # would be 17k if we carried out no preprocessing

10329

In [46]:
model.wv['dog']

array([ 0.12672846,  0.02294515,  0.2559114 , -0.3159711 ,  0.08723502,
       -0.25824797, -0.13400078,  0.23638535, -0.3706651 ,  0.08617202,
       -0.02608794, -0.07779571, -0.1575244 , -0.15905625, -0.09371224,
       -0.31183934,  0.04625181,  0.3393061 , -0.49791494,  0.24282694,
        0.12577783,  0.31358212,  0.3264093 , -0.13702764, -0.37694553,
        0.22277185, -0.61593   , -0.05588112,  0.22323394,  0.0428558 ,
        0.16754407, -0.140997  , -0.08871103, -0.19913545,  0.26169133,
       -0.32252732,  0.26109895, -0.16887061, -0.29031727,  0.33175352,
        0.00472591,  0.22646251, -0.01365729, -0.350554  , -0.3846127 ,
        0.1558706 , -0.3502834 ,  0.05032256, -0.24653903,  0.17062403,
        0.13588569,  0.8472679 ,  0.51008487, -0.11496582,  0.52628154,
        0.49547285,  0.184831  ,  0.0124798 , -0.2652375 , -0.2033405 ,
        0.27597156, -0.3773898 , -0.50599915, -0.24444523], dtype=float32)

In [48]:
len(model.wv['dog'])

64

In [49]:
model.wv.most_similar('dog', topn=3)

[('puppy', 0.8074982166290283),
 ('cage', 0.7512398362159729),
 ('cow', 0.751176655292511)]

In [50]:
model.wv.most_similar('eat', topn=3)

[('bread', 0.8535346388816833),
 ('drink', 0.8206859230995178),
 ('meat', 0.7800751328468323)]

In [51]:
model.wv.most_similar('day', topn=3)

[('morning', 0.7905176281929016),
 ('night', 0.750615119934082),
 ('week', 0.7250584363937378)]

In [52]:
model.wv.most_similar('father', topn=3)

[('mother', 0.8100275993347168),
 ('brother', 0.7530925273895264),
 ('dearly', 0.749744176864624)]

In [53]:
model.wv.most_similar('ma_am', topn=3)

[('betty', 0.8617087006568909),
 ('mamma', 0.8547512888908386),
 ('madam', 0.8509178161621094)]

In [54]:
model.wv.doesnt_match("mother father sister brother dog".split())

'dog'

In [55]:
model.wv.similarity('father', 'dog')

0.49985236

In [56]:
model.wv.most_similar(positive=['father', 'woman'], negative=['man'])

[('sister', 0.78533935546875),
 ('mother', 0.7696622014045715),
 ('husband', 0.7475460767745972),
 ('daughter', 0.747188150882721),
 ('wife', 0.7430208325386047),
 ('elder', 0.7088102102279663),
 ('sarah', 0.6988385915756226),
 ('rebekah', 0.6969820857048035),
 ('nurse', 0.6940966248512268),
 ('loved', 0.6882919073104858)]

In [57]:
model.wv.most_similar(positive=['husband', 'woman'], negative=['man'])

[('wife', 0.7246286869049072),
 ('sister', 0.7216459512710571),
 ('daughter', 0.682881236076355),
 ('maid', 0.666713297367096),
 ('conceived', 0.6662355661392212),
 ('mother', 0.6650395393371582),
 ('elder', 0.6436616778373718),
 ('child', 0.6310928463935852),
 ('nurse', 0.6217225193977356),
 ('daughters', 0.599367082118988)]

## Reduce Word Vector Dimensionality With t-sne

In [61]:
tsne = TSNE(n_components=2, n_iter=1000)

In [63]:
X_2d = tsne.fit_transform(model.wv[model.wv.key_to_index])

In [64]:
coord_df = pd.DataFrame(X_2d, columns=['x', 'y'])
coord_df['token'] = model.wv.key_to_index

In [65]:
coord_df.head(5)

,x,y,token
0,-28.758465,42.990028,the
1,-28.892147,43.010056,and
2,-28.765133,43.303837,of
3,-29.208563,42.742439,to
4,-28.946939,41.170715,a


In [66]:
coord_df.to_csv('clean_gutenberg_tsne.csv', index=False)

## Visualise

In [67]:
coord_df = pd.read_csv('clean_gutenberg_tsne.csv')


In [68]:
output_notebook()

Loading BokehJS ...

In [69]:
subset_df = coord_df.sample(n=5000)

In [70]:
p = figure(plot_width=800, plot_height=800)
_ = p.text(x=subset_df.x, y=subset_df.y, text=subset_df.token)

In [71]:
show(p)